In [1]:
path=  'C:/Users/felipe/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/'
#import sys
#sys.path.append(path) 
path_=  'C:/Users/felipe/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/Brain_Age_Clock/Base_Models'
import sys
sys.path.append(path_) 

# Regresor

In [2]:
from sklearn.linear_model import ElasticNet
from base_regressor import BaseRegressor
from Plotter import Plotter
from skopt.space import Real, Categorical, Integer

class ElasticNetRegressor(BaseRegressor):
    def __init__(self, save_path=None, scaler=None, params=None, params_space=None, fit_params_search=None, model_params_search=None, fit_params_train=None, model_params_train=None, name_model="ElasticNet"):
        super().__init__(save_path, scaler, params, params_space, fit_params_search, model_params_search, fit_params_train, model_params_train, name_model)

        self.model_ml = ElasticNet
        if params is None:
            self.params = {
                'alpha': 0.2,
                'l1_ratio': 0.5,  # Proporción de L1 en la regularización
                'max_iter': 10000,
                # 'tol': 0.001
            }
        if params_space is None:
            self.params_space = {
                'alpha': Real(0.001, 0.5, prior='log-uniform'),
                'l1_ratio': Real(0.0, 1.0),  # Rango de 0 a 1 para la proporción de L1
                'max_iter': Integer(100, 1000),
                'tol': Real(1e-5, 1e-2, prior='log-uniform')
            }


c:\python_env\brain_age_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Clasificador

In [3]:
from base_classifier import BaseClassifier
from skopt.space import Real, Categorical, Integer
import xgboost as xgb

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold


class XGBoostClassifier(BaseClassifier):
    def __init__(self, save_path=None, scaler=None, params=None, params_space=None, fit_params_search=None, model_params_search=None, fit_params_train=None, model_params_train=None, name_model="Lasso"):
        super().__init__(save_path, scaler, params, params_space, fit_params_search, model_params_search, fit_params_train, model_params_train, name_model)

        self.model_ml = xgb.XGBClassifier
        if params is None:
            self.params = {
            'objective': 'binary:logistic',  # Clasificación binaria
            'eval_metric': 'logloss',       # Métrica de evaluación
            'learning_rate': 0.1,           # Tasa de aprendizaje
            'max_depth': 6,                 # Profundidad máxima del árbol
            'subsample': 0.8,               # Proporción de muestras utilizadas por árbol
            'colsample_bytree': 0.8,        # Proporción de características utilizadas por árbol
            'reg_alpha': 0.1,               # Regularización L1
            'reg_lambda': 1.0,              # Regularización L2
            'gamma': 0.0,                   # Mínima reducción de pérdida para crear una división
            'min_child_weight': 1,          # Mínimo peso de la suma de los hijos para una división
            'n_estimators': 100,            # Número de árboles en el modelo
            'scale_pos_weight': 1.0         # Para datos desbalanceados
        }
        if params_space is None:
            self.params_space = {
                'objective'   : ['binary:logistic'],
                'eval_metric' : ['logloss', 'error', 'auc', 'aucpr'],
                'learning_rate': (0.01, 0.1, 0.2),
                'max_depth': (3, 4, 5),
                'subsample': (0.8, 0.9, 1.0),
                'colsample_bytree': (0.8, 0.9, 1.0),
                'reg_alpha': (0, 0.1, 1.0),
                'reg_lambda': (0, 0.1, 1.0),
                #'gamma': (0.001, 0.01, 0.1, 1, 10),
            }

# Instancia de modelos 

In [4]:
from Plotter import Plotter
from sklearn.preprocessing import MinMaxScaler,StandardScaler

model_reg = ElasticNetRegressor()

Plotters = Plotter()

# Parametros de Plot
colorset = 'darkorange'
nameset = 'ElasticNet'

#parametros de scaler
#1:sin scaler 2:Zscore 3:MinMax
Scaler_reg_train=2

#scaler = MinMaxScaler()
Scaler_reg = StandardScaler()

model_cls = XGBoostClassifier()


In [5]:
#72 features
features = ['Alpha2_canon_EPP_HPC_left', 'Alpha2_canon_EPP_HPC_right', 'Alpha2_canon_EPP_PARIET_left', 'Alpha2_canon_EPP_PARIET_right', 'Alpha2_canon_EPP_OCC_left', 'Alpha2_canon_EPP_OCC_right', 'Alpha2_canon_RPD_HPC_left', 'Alpha2_canon_RPD_HPC_right', 'Alpha2_canon_RPD_CING_left', 'Alpha2_canon_RPD_CING_right', 'Alpha2_canon_RPD_PARIET_left', 'Alpha2_canon_RPD_PARIET_right', 'Alpha2_canon_RPD_OCC_left', 'Alpha2_canon_RPD_OCC_right', 'High_subj_spec_EPP_HPC_left', 'High_subj_spec_EPP_HPC_right', 'High_subj_spec_EPP_CING_left', 'High_subj_spec_EPP_CING_right', 'High_subj_spec_EPP_PARIET_left', 'High_subj_spec_EPP_PARIET_right', 'High_subj_spec_EPP_OCC_left', 'High_subj_spec_EPP_OCC_right', 'High_subj_spec_RPD_HPC_left', 'High_subj_spec_RPD_HPC_right', 'High_subj_spec_RPD_CING_left', 'High_subj_spec_RPD_CING_right', 'High_subj_spec_RPD_PARIET_left', 'High_subj_spec_RPD_PARIET_right', 'High_subj_spec_RPD_OCC_left', 'High_subj_spec_RPD_OCC_right', 'Low_subj_spec_EPP_HPC_left', 'Low_subj_spec_EPP_HPC_right', 'Low_subj_spec_EPP_CING_left', 'Low_subj_spec_EPP_CING_right', 'Low_subj_spec_EPP_PARIET_left', 'Low_subj_spec_EPP_PARIET_right', 'Low_subj_spec_EPP_OCC_left', 'Low_subj_spec_EPP_OCC_right', 'Low_subj_spec_RPD_HPC_left', 'Low_subj_spec_RPD_HPC_right', 'Low_subj_spec_RPD_CING_left', 'Low_subj_spec_RPD_CING_right', 'Low_subj_spec_RPD_PARIET_left', 'Low_subj_spec_RPD_PARIET_right', 'Low_subj_spec_RPD_OCC_left', 'Low_subj_spec_RPD_OCC_right', 'TF_ORB_left', 'TF_ORB_right', 'TF_IFG_left', 'TF_IFG_right', 'TF_MFG_left', 'TF_MFG_right', 'TF_SFG_left', 'TF_SFG_right', 'TF_HPC_left', 'TF_HPC_right', 'TF_INS_left', 'TF_INS_right', 'IAF_ORB_left', 'IAF_ORB_right', 'IAF_IFG_left', 'IAF_IFG_right', 'IAF_MFG_left', 'IAF_MFG_right', 'IAF_SFG_left', 'IAF_SFG_right', 'IAF_HPC_left', 'IAF_HPC_right', 'IAF_INS_left', 'IAF_INS_right', 'IAF_OCC_left', 'IAF_OCC_right']

# Data

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import pickle


file_path_CN = f'{path}Brain_Age_Clock/data_freq/CN_freq.xlsx'
file_path_AD = f'{path}Brain_Age_Clock/data_freq/AD_freq.xlsx'
file_path_FTD = f'{path}Brain_Age_Clock/data_freq/FTD_freq.xlsx'
file_path_MCI = f'{path}Brain_Age_Clock/data_freq/MCI_freq.xlsx'

df_CN = pd.read_excel(file_path_CN)
df_AD = pd.read_excel(file_path_AD)
df_FTD = pd.read_excel(file_path_FTD)
df_MCI = pd.read_excel(file_path_MCI)

df_CN_filtrado = df_CN[(df_CN['Age'] >=50) & (df_CN['Age'] <= 90)].reset_index(drop=True)
df_AD_filtrado = df_AD[(df_AD['Age'] >= 50) & (df_AD['Age'] <= 90)].reset_index(drop=True)
df_FTD_filtrado = df_FTD[(df_FTD['Age'] >= 50) & (df_FTD['Age'] <= 90)].reset_index(drop=True)
df_MCI_filtrado = df_MCI[(df_MCI['Age'] >= 50) & (df_MCI['Age'] <= 90)].reset_index(drop=True)


In [13]:
X_AD = df_AD_filtrado[features]
y_AD = df_AD_filtrado["Age"]
ID_AD = df_AD_filtrado["ID_unique"]

X_FTD = df_FTD_filtrado[features]
y_FTD = df_FTD_filtrado["Age"]
ID_FTD = df_FTD_filtrado["ID_unique"]

X_MCI = df_MCI_filtrado[features]
y_MCI = df_MCI_filtrado["Age"]
ID_MCI = df_MCI_filtrado["ID_unique"]

In [55]:
# Definir las regiones para cada subconjunto
regiones_1 = [ "Turquia","Europa", "America"]
regiones_2 = ["Turquia","Europa", "America"]

# Filtrar las filas del dataframe por región
df_CN_1 = df_CN_filtrado[df_CN_filtrado["Region"].isin(regiones_1)]
df_CN_2 = df_CN_filtrado[df_CN_filtrado["Region"].isin(regiones_2)]

# Seleccionar las columnas correspondientes para cada conjunto
X_CN_1 = df_CN_1[features]
y_CN_1 = df_CN_1["Age"]
ID_CN_1 = df_CN_1["ID_unique"]


X_CN_2 = df_CN_2[features]
y_CN_2 = df_CN_2["Age"]
ID_CN_2 = df_CN_2["ID_unique"]

scaler = StandardScaler()
scaler.fit(X_CN_1)

X_CN_1_scaled = scaler.transform(X_CN_1)
X_CN_1_scaled = pd.DataFrame(X_CN_1_scaled, columns=X_CN_1.columns)

df_concatenado_CN = pd.concat([X_CN_1, y_CN_1, ID_CN_1], axis=1, ignore_index=False)

In [56]:
best_params = {'alpha': 0.1,
 'l1_ratio': 0.1,
 'max_iter': 1000,
 'tol': 0.01}

In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

results = []

# Iterar por cada característica
for feature in features:
    # Crear datasets con solo la característica actual y la columna objetivo
    X_CN_ = X_CN_1[[feature]]
    
    # Dividir en train/test
    #opt_model, best_params = model_reg.search_best_model (X=X_CN_, y=y_CN_1, n_iter_=10, scoring_metric='r2')
    
    

    # Entrenar modelo (ejemplo con regresión lineal)
    #model = ...  # Tu modelo aquí
    results_labels_df_CN_train, results_labels_df_CN_test,  results_model, results_per_fold_CN_train, results_per_fold_CN_test, df_CN_avg_train,df_CN_avg_test= model_reg.trainer(
        X=X_CN_,
        y=y_CN_1,
        ID_label='ID_unique',
        ID=ID_CN_1,
        n_splits=10,
        n_iterations=20,
        params_=best_params
    )

    

    results_labels_df_AD_test, results_per_fold_AD_test, df_AD_avg_test = model_reg.test(X=X_AD[[feature]], y=y_AD, ID= ID_AD, ID_label='ID_unique', n_splits=10, n_iterations=20,result_model=results_model)
    results_labels_df_FTD_test, results_per_fold_FTD_test, df_FTD_avg_test = model_reg.test(X=X_FTD[[feature]], y=y_FTD, ID= ID_FTD, ID_label='ID_unique', n_splits=10, n_iterations=20,result_model=results_model)
    results_labels_df_MCI_test, results_per_fold_MCI_test, df_MCI_avg_test = model_reg.test(X=X_MCI[[feature]], y=y_MCI, ID= ID_MCI, ID_label='ID_unique', n_splits=10, n_iterations=20,result_model=results_model)



    df_CN_1_seleccion = df_CN_filtrado[['ID_unique', 'Country_unique','Region', 'Age', 'Sex', 'Educ', 'MMSE', 'HDI', 'GINI', 'ODQ']]
    df_CN_1_combined = df_CN_avg_test.merge(df_CN_1_seleccion, on='ID_unique', how='left')

    df_AD_seleccion = df_AD_filtrado[['ID_unique', 'Country_unique','Region', 'Age', 'Sex', 'Educ', 'MMSE',  'GINI', 'ODQ']]
    df_AD_combined = df_AD_avg_test.merge(df_AD_seleccion, on='ID_unique', how='left')

    df_FTD_seleccion = df_FTD_filtrado[['ID_unique', 'Country_unique','Region', 'Age', 'Sex', 'Educ', 'MMSE', 'GINI', 'ODQ']]
    df_FTD_combined = df_FTD_avg_test.merge(df_FTD_seleccion, on='ID_unique', how='left')

    df_MCI_seleccion = df_MCI_filtrado[['ID_unique', 'Country_unique', 'Region','Age', 'Sex', 'Educ', 'MMSE',  'GINI', 'ODQ']]
    df_MCI_combined = df_MCI_avg_test.merge(df_MCI_seleccion, on='ID_unique', how='left')

  
    # Calcular métricas en el train set
    r2 = r2_score(df_CN_avg_train['y_labels'], df_CN_avg_train['y_pred'])
    mae = mean_absolute_error(df_CN_avg_train['y_labels'], df_CN_avg_train['y_pred'])
    


    # Calcular diferencias GAP promediada

    avg_gap_CN = np.mean(df_CN_avg_test['GAP_corrected'])

    avg_gap_AD = np.mean(df_AD_avg_test['GAP_corrected']) - np.mean(df_CN_avg_test['GAP_corrected'])
    avg_gap_FTD = np.mean(df_FTD_avg_test['GAP_corrected']) - np.mean(df_CN_avg_test['GAP_corrected'])
    avg_gap_MCI = np.mean(df_MCI_avg_test['GAP_corrected']) - np.mean(df_CN_avg_test['GAP_corrected'])
    
    avg_gap_CN_America = np.mean(np.mean(df_CN_1_combined[df_CN_1_combined['Region'] == 'America']['GAP_corrected']))
    avg_gap_CN_Europa = np.mean(np.mean(df_CN_1_combined[df_CN_1_combined['Region'] == 'Europa']['GAP_corrected']))
    avg_gap_CN_Turquia = np.mean(np.mean(df_CN_1_combined[df_CN_1_combined['Region'] == 'Turquia']['GAP_corrected']))

    avg_gap_AD_America = np.mean(df_AD_combined[df_AD_combined['Region'] == 'America']['GAP_corrected']) - np.mean(df_CN_1_combined[df_CN_1_combined['Region'] == 'America']['GAP_corrected'])
    avg_gap_AD_Europa = np.mean(df_AD_combined[df_AD_combined['Region'] == 'Europa']['GAP_corrected']) - np.mean(df_CN_1_combined[df_CN_1_combined['Region'] == 'Europa']['GAP_corrected'])
    avg_gap_AD_Turquia = np.mean(df_AD_combined[df_AD_combined['Region'] == 'Turquia']['GAP_corrected']) - np.mean(df_CN_1_combined[df_CN_1_combined['Region'] == 'Turquia']['GAP_corrected'])

    avg_gap_AD_America_rel = np.mean(df_AD_combined[df_AD_combined['Region'] == 'America']['GAP_corrected']) - np.mean(df_CN_avg_test['GAP_corrected'])
    avg_gap_AD_Europa_rel = np.mean(df_AD_combined[df_AD_combined['Region'] == 'Europa']['GAP_corrected']) - np.mean(df_CN_avg_test['GAP_corrected'])
    avg_gap_AD_Turquia_rel = np.mean(df_AD_combined[df_AD_combined['Region'] == 'Turquia']['GAP_corrected']) - np.mean(df_CN_avg_test['GAP_corrected'])


    # Guardar resultados
    results.append({
        "Feature": feature,
        "R2": r2,
        "MAE": mae,
        "Avg_GAP_CN": avg_gap_CN,
        "Avg_GAP_AD": avg_gap_AD,
        "Avg_GAP_FTD": avg_gap_FTD,
        "Avg_GAP_MCI": avg_gap_MCI,
        "Avg_GAP_CN_America": avg_gap_CN_America,
        "Avg_GAP_CN_Europa": avg_gap_CN_Europa,
        "Avg_GAP_CN_Turquia": avg_gap_CN_Turquia,
        "Avg_GAP_AD_America": avg_gap_AD_America,
        "Avg_GAP_AD_Europa": avg_gap_AD_Europa,
        "Avg_GAP_AD_Turquia": avg_gap_AD_Turquia,
        "Avg_GAP_AD_America_Rel": avg_gap_AD_America_rel,
        "Avg_GAP_AD_Europa_Rel": avg_gap_AD_Europa_rel,
        "Avg_GAP_AD_Turquia_Rel": avg_gap_AD_Turquia_rel,
    })


# Convertir resultados a dataframe
results_df = pd.DataFrame(results)

# Ordenar por la métrica que te interese, por ejemplo, menor MAE
results_df = results_df.sort_values(by="MAE")

C:\Users/felipe/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/Brain_Age_Clock/Base_Models\base_regressor.py:207: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_labels_df_train = pd.concat([results_labels_df_train, temp_df_train], ignore_index=True)
C:\Users/felipe/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/Brain_Age_Clock/Base_Models\base_regressor.py:209: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_labels_df_test = pd.concat([res

In [ ]:
results_df = results_df.sort_values(by="MAE")

In [58]:
results_df

,Feature,R2,MAE,Avg_GAP_CN,Avg_GAP_AD,Avg_GAP_FTD,Avg_GAP_MCI,Avg_GAP_CN_America,Avg_GAP_CN_Europa,Avg_GAP_CN_Turquia,Avg_GAP_AD_America,Avg_GAP_AD_Europa,Avg_GAP_AD_Turquia,Avg_GAP_AD_America_Rel,Avg_GAP_AD_Europa_Rel,Avg_GAP_AD_Turquia_Rel
38,Low_subj_spec_RPD_HPC_left,0.023977,6.621206,0.000258,0.446288,0.498145,0.450691,-0.257328,0.506111,-0.118776,1.072175,NaN,-0.232664,0.814590,NaN,-0.351698
2,Alpha2_canon_EPP_PARIET_left,0.019487,6.624715,0.002638,0.494198,0.464841,0.417073,-0.521977,0.333567,0.262498,1.162297,NaN,-0.076544,0.637682,NaN,0.183316
10,Alpha2_canon_RPD_PARIET_left,0.019487,6.624715,0.002638,0.494198,0.464841,0.417073,-0.521977,0.333567,0.262498,1.162297,NaN,-0.076544,0.637682,NaN,0.183316
22,High_subj_spec_RPD_HPC_left,0.022320,6.627465,0.000892,0.353321,0.366469,0.403222,-0.289798,0.475004,-0.063947,0.931910,NaN,-0.205622,0.641221,NaN,-0.270461
14,High_subj_spec_EPP_HPC_left,0.022320,6.627465,0.000892,0.353321,0.366469,0.403222,-0.289798,0.475004,-0.063947,0.931910,NaN,-0.205622,0.641221,NaN,-0.270461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,Low_subj_spec_EPP_HPC_left,0.000621,6.738101,0.001507,0.003207,0.005776,-0.007104,0.014012,-0.006328,-0.004725,-0.004905,NaN,-0.000079,0.007600,NaN,-0.006311
64,IAF_SFG_left,0.000851,6.738324,0.000057,0.011707,0.035541,0.013813,0.026274,0.013751,-0.034643,0.009689,NaN,-0.006024,0.035906,NaN,-0.040724
37,Low_subj_spec_EPP_OCC_right,0.000637,6.739191,0.001502,0.016519,-0.004348,0.007768,0.026524,-0.019894,-0.006862,-0.007422,NaN,0.022542,0.017600,NaN,0.014178
54,TF_HPC_left,0.001030,6.739352,0.001061,0.052918,0.031359,0.023183,0.033540,-0.021828,-0.013303,0.038746,NaN,0.027615,0.071225,NaN,0.013251


In [59]:
results_df.to_excel("results_metrics_all_sin_cuba.xlsx", index=False)

In [17]:
import pandas as pd

# Definir las condiciones para filtrar las filas
condiciones = (
    (results_df['Avg_GAP_AD'] == results_df['Avg_GAP_AD'].max()) &
    (results_df['Avg_GAP_CN_America'] == results_df['Avg_GAP_CN_America'].min()) &
    (results_df['Avg_GAP_CN_Europa'] == results_df['Avg_GAP_CN_Europa'].min()) &
    (results_df['Avg_GAP_CN_Turquia'] == results_df['Avg_GAP_CN_Turquia'].min()) &
    (results_df['Avg_GAP_AD_America'] == results_df['Avg_GAP_AD_America'].max()) &
    (results_df['Avg_GAP_AD_Turquia'] == results_df['Avg_GAP_AD_Turquia'].max()) &
    (results_df['Avg_GAP_AD_America_Rel'] == results_df['Avg_GAP_AD_America_Rel'].max()) &
    (results_df['Avg_GAP_AD_Turquia_Rel'] == results_df['Avg_GAP_AD_Turquia_Rel'].max())
)

# Filtrar el DataFrame según las condiciones
caracteristicas_seleccionadas = results_df[condiciones]

# Mostrar las características seleccionadas
print(caracteristicas_seleccionadas)


Empty DataFrame
Columns: [Feature, R2, MAE, Avg_GAP_CN, Avg_GAP_AD, Avg_GAP_FTD, Avg_GAP_MCI, Avg_GAP_CN_America, Avg_GAP_CN_Europa, Avg_GAP_CN_Turquia, Avg_GAP_AD_America, Avg_GAP_AD_Europa, Avg_GAP_AD_Turquia, Avg_GAP_AD_America_Rel, Avg_GAP_AD_Europa_Rel, Avg_GAP_AD_Turquia_Rel]
Index: []


In [30]:
# Ordenar por las columnas de interés según tus criterios
results_filtered = results_df.copy()

# Filtrar las filas con las mejores características de acuerdo con tus criterios
# El criterio es que algunas columnas deben ser máximas y otras mínimas, así que lo ordenamos en consecuencia.
results_filtered = results_filtered.sort_values(
    by=[
        "Avg_GAP_AD",  # Lo más alto posible
        "Avg_GAP_CN_America",  # Lo más bajo posible
        "Avg_GAP_CN_Europa",  # Lo más bajo posible
        "Avg_GAP_CN_Turquia",  # Lo más bajo posible
        "Avg_GAP_AD_America",  # Lo más alto posible
        "Avg_GAP_AD_Turquia",  # Lo más alto posible
        "Avg_GAP_AD_America_Rel",  # Lo más alto posible
        "Avg_GAP_AD_Turquia_Rel"  # Lo más alto posible
    ],
    ascending=[
        False,  # Para que Avg_GAP_AD sea lo más alto posible
        True,   # Para que Avg_GAP_CN_America sea lo más bajo posible
        True,   # Para que Avg_GAP_CN_Europa sea lo más bajo posible
        True,   # Para que Avg_GAP_CN_Turquia sea lo más bajo posible
        False,  # Para que Avg_GAP_AD_America sea lo más alto posible
        False,  # Para que Avg_GAP_AD_Turquia sea lo más alto posible
        False,  # Para que Avg_GAP_AD_America_Rel sea lo más alto posible
        False   # Para que Avg_GAP_AD_Turquia_Rel sea lo más alto posible
    ]
)

# Filtrar las mejores características según el orden (puedes elegir el número de mejores características a seleccionar)
top_features = results_filtered.head(50)  # Ajusta el número 10 a cuántas características quieres obtener

print(top_features)


                            Feature        R2       MAE  Avg_GAP_CN  \
43   Low_subj_spec_RPD_PARIET_right  0.016324  6.678903    0.001908   
19  High_subj_spec_EPP_PARIET_right  0.014659  6.686423    0.001802   
27  High_subj_spec_RPD_PARIET_right  0.014659  6.686423    0.001802   
39      Low_subj_spec_RPD_HPC_right  0.020969  6.671373    0.001734   
42    Low_subj_spec_RPD_PARIET_left  0.011446  6.699834    0.002446   
11    Alpha2_canon_RPD_PARIET_right  0.013375  6.680971    0.002273   
3     Alpha2_canon_EPP_PARIET_right  0.013375  6.680971    0.002273   
40      Low_subj_spec_RPD_CING_left  0.010782  6.710088    0.004206   
8        Alpha2_canon_RPD_CING_left  0.012284  6.687464    0.002500   
10     Alpha2_canon_RPD_PARIET_left  0.014918  6.668156    0.002503   
2      Alpha2_canon_EPP_PARIET_left  0.014918  6.668156    0.002503   
41     Low_subj_spec_RPD_CING_right  0.008251  6.729478    0.003109   
16     High_subj_spec_EPP_CING_left  0.011391  6.700551    0.003710   
24    

In [31]:
top_features['Feature']

43     Low_subj_spec_RPD_PARIET_right
19    High_subj_spec_EPP_PARIET_right
27    High_subj_spec_RPD_PARIET_right
39        Low_subj_spec_RPD_HPC_right
42      Low_subj_spec_RPD_PARIET_left
11      Alpha2_canon_RPD_PARIET_right
3       Alpha2_canon_EPP_PARIET_right
40        Low_subj_spec_RPD_CING_left
8          Alpha2_canon_RPD_CING_left
10       Alpha2_canon_RPD_PARIET_left
2        Alpha2_canon_EPP_PARIET_left
41       Low_subj_spec_RPD_CING_right
16       High_subj_spec_EPP_CING_left
24       High_subj_spec_RPD_CING_left
26     High_subj_spec_RPD_PARIET_left
18     High_subj_spec_EPP_PARIET_left
15       High_subj_spec_EPP_HPC_right
23       High_subj_spec_RPD_HPC_right
38         Low_subj_spec_RPD_HPC_left
45        Low_subj_spec_RPD_OCC_right
44         Low_subj_spec_RPD_OCC_left
17      High_subj_spec_EPP_CING_right
25      High_subj_spec_RPD_CING_right
9         Alpha2_canon_RPD_CING_right
1          Alpha2_canon_EPP_HPC_right
7          Alpha2_canon_RPD_HPC_right
14        Hi

In [32]:

# Convertir la columna 'Feature' de top_features a una lista
selected_features_list = top_features['Feature'].tolist()

print(selected_features_list)

['Low_subj_spec_RPD_PARIET_right', 'High_subj_spec_EPP_PARIET_right', 'High_subj_spec_RPD_PARIET_right', 'Low_subj_spec_RPD_HPC_right', 'Low_subj_spec_RPD_PARIET_left', 'Alpha2_canon_RPD_PARIET_right', 'Alpha2_canon_EPP_PARIET_right', 'Low_subj_spec_RPD_CING_left', 'Alpha2_canon_RPD_CING_left', 'Alpha2_canon_RPD_PARIET_left', 'Alpha2_canon_EPP_PARIET_left', 'Low_subj_spec_RPD_CING_right', 'High_subj_spec_EPP_CING_left', 'High_subj_spec_RPD_CING_left', 'High_subj_spec_RPD_PARIET_left', 'High_subj_spec_EPP_PARIET_left', 'High_subj_spec_EPP_HPC_right', 'High_subj_spec_RPD_HPC_right', 'Low_subj_spec_RPD_HPC_left', 'Low_subj_spec_RPD_OCC_right', 'Low_subj_spec_RPD_OCC_left', 'High_subj_spec_EPP_CING_right', 'High_subj_spec_RPD_CING_right', 'Alpha2_canon_RPD_CING_right', 'Alpha2_canon_EPP_HPC_right', 'Alpha2_canon_RPD_HPC_right', 'High_subj_spec_EPP_HPC_left', 'High_subj_spec_RPD_HPC_left', 'High_subj_spec_RPD_OCC_left', 'High_subj_spec_EPP_OCC_left', 'High_subj_spec_EPP_OCC_right', 'High_su

In [66]:
import pandas as pd

# Copiar el DataFrame original para no modificarlo
results_filtered = results_df.copy()

# Inicializar una columna de puntajes
results_filtered['Score'] = 0

# Maximizar o minimizar columnas según los criterios, asignando puntajes
results_filtered['Score'] += results_filtered['Avg_GAP_AD']  # Maximizar Avg_GAP_AD (lo más alto posible)
#results_filtered['Score'] -= results_filtered['Avg_GAP_CN_America']  # Minimizar Avg_GAP_CN_America (lo más bajo posible)
results_filtered['Score'] -= results_filtered['Avg_GAP_CN_Europa']  # Minimizar Avg_GAP_CN_Europa (lo más bajo posible)
#results_filtered['Score'] -= results_filtered['Avg_GAP_CN_Turquia']  # Minimizar Avg_GAP_CN_Turquia (lo más bajo posible)
results_filtered['Score'] += results_filtered['Avg_GAP_AD_America']  # Maximizar Avg_GAP_AD_America (lo más alto posible)
results_filtered['Score'] += results_filtered['Avg_GAP_AD_Turquia']  # Maximizar Avg_GAP_AD_Turquia (lo más alto posible)
#results_filtered['Score'] += results_filtered['Avg_GAP_AD_America_Rel']  # Maximizar Avg_GAP_AD_America_Rel (lo más alto posible)
#results_filtered['Score'] += results_filtered['Avg_GAP_AD_Turquia_Rel']  # Maximizar Avg_GAP_AD_Turquia_Rel (lo más alto posible)

# Ordenar las características según su puntaje (el puntaje más alto es el mejor)
results_filtered = results_filtered.sort_values(by='Score', ascending=False)

# Seleccionar las 10 características con el puntaje más alto
top_features = results_filtered.head(50)

# Convertir la columna 'Feature' de top_features a una lista
selected_features_list = top_features['Feature'].tolist()

# Mostrar la lista de características seleccionadas
print(selected_features_list)


['Low_subj_spec_RPD_PARIET_right', 'High_subj_spec_EPP_PARIET_right', 'High_subj_spec_RPD_PARIET_right', 'Low_subj_spec_RPD_PARIET_left', 'Low_subj_spec_RPD_CING_left', 'High_subj_spec_EPP_CING_left', 'High_subj_spec_RPD_CING_left', 'High_subj_spec_RPD_PARIET_left', 'High_subj_spec_EPP_PARIET_left', 'Alpha2_canon_RPD_PARIET_right', 'Alpha2_canon_EPP_PARIET_right', 'Alpha2_canon_RPD_PARIET_left', 'Alpha2_canon_EPP_PARIET_left', 'Low_subj_spec_RPD_CING_right', 'Alpha2_canon_RPD_CING_left', 'Low_subj_spec_RPD_HPC_right', 'High_subj_spec_EPP_CING_right', 'High_subj_spec_RPD_CING_right', 'High_subj_spec_EPP_HPC_right', 'High_subj_spec_RPD_HPC_right', 'Low_subj_spec_RPD_OCC_right', 'Low_subj_spec_RPD_OCC_left', 'Alpha2_canon_RPD_CING_right', 'High_subj_spec_EPP_OCC_right', 'High_subj_spec_RPD_OCC_right', 'Low_subj_spec_RPD_HPC_left', 'High_subj_spec_RPD_OCC_left', 'High_subj_spec_EPP_OCC_left', 'Alpha2_canon_EPP_HPC_right', 'Alpha2_canon_RPD_HPC_right', 'High_subj_spec_EPP_HPC_left', 'High_s

In [65]:
results_filtered

,Feature,R2,MAE,Avg_GAP_CN,Avg_GAP_AD,Avg_GAP_FTD,Avg_GAP_MCI,Avg_GAP_CN_America,Avg_GAP_CN_Europa,Avg_GAP_CN_Turquia,Avg_GAP_AD_America,Avg_GAP_AD_Europa,Avg_GAP_AD_Turquia,Avg_GAP_AD_America_Rel,Avg_GAP_AD_Europa_Rel,Avg_GAP_AD_Turquia_Rel,Score
43,Low_subj_spec_RPD_PARIET_right,0.019092,6.641845,0.001768,0.680793,0.881532,0.380040,-0.391869,0.329412,0.139768,1.353050,NaN,-0.060882,0.959413,NaN,0.077117,1.643549
19,High_subj_spec_EPP_PARIET_right,0.018882,6.641400,0.002163,0.596717,0.832560,0.364670,-0.429566,0.358041,0.156009,1.279409,NaN,-0.100882,0.847680,NaN,0.052964,1.417203
27,High_subj_spec_RPD_PARIET_right,0.018882,6.641400,0.002163,0.596717,0.832560,0.364670,-0.429566,0.358041,0.156009,1.279409,NaN,-0.100882,0.847680,NaN,0.052964,1.417203
42,Low_subj_spec_RPD_PARIET_left,0.016585,6.653309,0.001979,0.579066,0.604149,0.351774,-0.421121,0.374308,0.135827,1.205778,NaN,0.004059,0.782677,NaN,0.137907,1.414595
40,Low_subj_spec_RPD_CING_left,0.013457,6.676595,0.001387,0.524489,0.357189,0.552121,-0.461403,0.281624,0.239023,1.105344,NaN,0.031047,0.642553,NaN,0.268682,1.379255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,TF_SFG_left,0.002229,6.733315,0.001751,-0.096569,-0.025930,-0.057147,-0.103223,0.039226,0.074394,-0.026605,NaN,-0.093356,-0.131579,NaN,-0.020714,-0.255755
53,TF_SFG_right,0.002724,6.734002,0.001635,-0.124180,-0.064017,-0.069980,-0.100919,0.033229,0.076208,-0.063433,NaN,-0.108168,-0.165988,NaN,-0.033596,-0.329009
33,Low_subj_spec_EPP_CING_right,0.004192,6.718174,0.000326,-0.104325,0.060622,0.004547,-0.162836,0.165903,0.036144,0.051352,NaN,-0.123927,-0.111809,NaN,-0.088109,-0.342803
56,TF_INS_left,0.002522,6.724813,0.000652,-0.141387,-0.071261,-0.049069,-0.095102,0.067139,0.043712,-0.085548,NaN,-0.097964,-0.181302,NaN,-0.054904,-0.392038


In [93]:
filtered_features_all = results_df.loc[(results_df['Avg_GAP_AD'] > 0.0) , 'Feature']

# Convertir a lista
filtered_features_list_all = filtered_features_all.tolist()

print(filtered_features_list_all)

['Low_subj_spec_RPD_HPC_left', 'Alpha2_canon_EPP_PARIET_left', 'Alpha2_canon_RPD_PARIET_left', 'High_subj_spec_RPD_HPC_left', 'High_subj_spec_EPP_HPC_left', 'High_subj_spec_EPP_PARIET_left', 'High_subj_spec_RPD_PARIET_left', 'Low_subj_spec_RPD_HPC_right', 'Alpha2_canon_EPP_PARIET_right', 'Alpha2_canon_RPD_PARIET_right', 'High_subj_spec_EPP_PARIET_right', 'High_subj_spec_RPD_PARIET_right', 'Low_subj_spec_RPD_PARIET_right', 'Alpha2_canon_RPD_HPC_left', 'Alpha2_canon_EPP_HPC_left', 'Low_subj_spec_RPD_PARIET_left', 'High_subj_spec_EPP_HPC_right', 'High_subj_spec_RPD_HPC_right', 'High_subj_spec_RPD_CING_left', 'High_subj_spec_EPP_CING_left', 'High_subj_spec_EPP_OCC_left', 'High_subj_spec_RPD_OCC_left', 'Low_subj_spec_RPD_OCC_left', 'Alpha2_canon_RPD_CING_left', 'Alpha2_canon_EPP_HPC_right', 'Alpha2_canon_RPD_HPC_right', 'Low_subj_spec_RPD_CING_left', 'Alpha2_canon_EPP_OCC_left', 'Alpha2_canon_RPD_OCC_left', 'Low_subj_spec_RPD_OCC_right', 'High_subj_spec_RPD_OCC_right', 'High_subj_spec_EPP_O

In [94]:
filtered_features_America = results_df.loc[ (results_df['Avg_GAP_AD_America'] > 0.0), 'Feature']

# Convertir a lista
filtered_features_list_America = filtered_features_America.tolist()

print(filtered_features_list_America)

['Low_subj_spec_RPD_HPC_left', 'Alpha2_canon_EPP_PARIET_left', 'Alpha2_canon_RPD_PARIET_left', 'High_subj_spec_RPD_HPC_left', 'High_subj_spec_EPP_HPC_left', 'High_subj_spec_EPP_PARIET_left', 'High_subj_spec_RPD_PARIET_left', 'Low_subj_spec_RPD_HPC_right', 'Alpha2_canon_EPP_PARIET_right', 'Alpha2_canon_RPD_PARIET_right', 'High_subj_spec_EPP_PARIET_right', 'High_subj_spec_RPD_PARIET_right', 'Low_subj_spec_RPD_PARIET_right', 'Alpha2_canon_RPD_HPC_left', 'Alpha2_canon_EPP_HPC_left', 'Low_subj_spec_RPD_PARIET_left', 'High_subj_spec_EPP_HPC_right', 'High_subj_spec_RPD_HPC_right', 'High_subj_spec_RPD_CING_left', 'High_subj_spec_EPP_CING_left', 'High_subj_spec_EPP_OCC_left', 'High_subj_spec_RPD_OCC_left', 'Low_subj_spec_RPD_OCC_left', 'Alpha2_canon_RPD_CING_left', 'Alpha2_canon_EPP_HPC_right', 'Alpha2_canon_RPD_HPC_right', 'Low_subj_spec_RPD_CING_left', 'Alpha2_canon_EPP_OCC_left', 'Alpha2_canon_RPD_OCC_left', 'Low_subj_spec_RPD_OCC_right', 'High_subj_spec_RPD_OCC_right', 'High_subj_spec_EPP_O

In [103]:
filtered_features_America_Rel = results_df.loc[ (results_df['Avg_GAP_AD_America_Rel'] > 0.0), 'Feature']

# Convertir a lista
filtered_features_list_America_Rel = filtered_features_America_Rel.tolist()

print(filtered_features_list_America_Rel)

['Low_subj_spec_RPD_HPC_left', 'Alpha2_canon_EPP_PARIET_left', 'Alpha2_canon_RPD_PARIET_left', 'High_subj_spec_RPD_HPC_left', 'High_subj_spec_EPP_HPC_left', 'High_subj_spec_EPP_PARIET_left', 'High_subj_spec_RPD_PARIET_left', 'Low_subj_spec_RPD_HPC_right', 'Alpha2_canon_EPP_PARIET_right', 'Alpha2_canon_RPD_PARIET_right', 'High_subj_spec_EPP_PARIET_right', 'High_subj_spec_RPD_PARIET_right', 'Low_subj_spec_RPD_PARIET_right', 'Alpha2_canon_RPD_HPC_left', 'Alpha2_canon_EPP_HPC_left', 'Low_subj_spec_RPD_PARIET_left', 'High_subj_spec_EPP_HPC_right', 'High_subj_spec_RPD_HPC_right', 'High_subj_spec_RPD_CING_left', 'High_subj_spec_EPP_CING_left', 'High_subj_spec_EPP_OCC_left', 'High_subj_spec_RPD_OCC_left', 'Low_subj_spec_RPD_OCC_left', 'Alpha2_canon_RPD_CING_left', 'Alpha2_canon_EPP_HPC_right', 'Alpha2_canon_RPD_HPC_right', 'Low_subj_spec_RPD_CING_left', 'Alpha2_canon_EPP_OCC_left', 'Alpha2_canon_RPD_OCC_left', 'Low_subj_spec_RPD_OCC_right', 'High_subj_spec_RPD_OCC_right', 'High_subj_spec_EPP_O

In [95]:
filtered_features_Turquia = results_df.loc[ (results_df['Avg_GAP_AD_Turquia'] > 0.0), 'Feature']

# Convertir a lista
filtered_features_list_Turquia = filtered_features_Turquia.tolist()

print(filtered_features_list_Turquia)

['Low_subj_spec_RPD_PARIET_left', 'Low_subj_spec_RPD_CING_left', 'High_subj_spec_EPP_CING_right', 'High_subj_spec_RPD_CING_right', 'Low_subj_spec_RPD_CING_right', 'IAF_OCC_left', 'TF_IFG_right', 'TF_ORB_right', 'TF_HPC_right', 'IAF_ORB_left', 'IAF_OCC_right', 'TF_INS_right', 'Low_subj_spec_EPP_OCC_right', 'TF_HPC_left']


In [104]:
filtered_features_Turquia_Rel = results_df.loc[ (results_df['Avg_GAP_AD_Turquia_Rel'] > 0.0), 'Feature']

# Convertir a lista
filtered_features_list_Turquia_Rel = filtered_features_Turquia_Rel.tolist()

print(filtered_features_list_Turquia_Rel)

['Alpha2_canon_EPP_PARIET_left', 'Alpha2_canon_RPD_PARIET_left', 'High_subj_spec_EPP_PARIET_left', 'High_subj_spec_RPD_PARIET_left', 'Alpha2_canon_EPP_PARIET_right', 'Alpha2_canon_RPD_PARIET_right', 'High_subj_spec_EPP_PARIET_right', 'High_subj_spec_RPD_PARIET_right', 'Low_subj_spec_RPD_PARIET_right', 'Low_subj_spec_RPD_PARIET_left', 'High_subj_spec_RPD_CING_left', 'High_subj_spec_EPP_CING_left', 'Alpha2_canon_RPD_CING_left', 'Low_subj_spec_RPD_CING_left', 'Alpha2_canon_RPD_CING_right', 'High_subj_spec_EPP_CING_right', 'High_subj_spec_RPD_CING_right', 'Low_subj_spec_RPD_CING_right', 'Alpha2_canon_RPD_OCC_right', 'Alpha2_canon_EPP_OCC_right', 'IAF_OCC_left', 'TF_HPC_right', 'IAF_ORB_left', 'IAF_OCC_right', 'Low_subj_spec_EPP_HPC_right', 'TF_MFG_left', 'Low_subj_spec_EPP_OCC_right', 'TF_HPC_left', 'TF_MFG_right']


In [111]:
filtered_features_R2 = results_df.loc[ (results_df['R2'] > 0.0), 'Feature']

# Convertir a lista
filtered_features_list_R2 = filtered_features_R2.tolist()

print(filtered_features_list_R2)

['Low_subj_spec_RPD_HPC_left', 'Alpha2_canon_EPP_PARIET_left', 'Alpha2_canon_RPD_PARIET_left', 'High_subj_spec_RPD_HPC_left', 'High_subj_spec_EPP_HPC_left', 'High_subj_spec_EPP_PARIET_left', 'High_subj_spec_RPD_PARIET_left', 'Low_subj_spec_RPD_HPC_right', 'Alpha2_canon_EPP_PARIET_right', 'Alpha2_canon_RPD_PARIET_right', 'High_subj_spec_EPP_PARIET_right', 'High_subj_spec_RPD_PARIET_right', 'Low_subj_spec_RPD_PARIET_right', 'Alpha2_canon_RPD_HPC_left', 'Alpha2_canon_EPP_HPC_left', 'Low_subj_spec_RPD_PARIET_left', 'High_subj_spec_EPP_HPC_right', 'High_subj_spec_RPD_HPC_right', 'High_subj_spec_RPD_CING_left', 'High_subj_spec_EPP_CING_left', 'High_subj_spec_EPP_OCC_left', 'High_subj_spec_RPD_OCC_left', 'Low_subj_spec_RPD_OCC_left', 'Alpha2_canon_RPD_CING_left', 'Alpha2_canon_EPP_HPC_right', 'Alpha2_canon_RPD_HPC_right', 'Low_subj_spec_RPD_CING_left', 'Alpha2_canon_EPP_OCC_left', 'Alpha2_canon_RPD_OCC_left', 'Low_subj_spec_RPD_OCC_right', 'High_subj_spec_RPD_OCC_right', 'High_subj_spec_EPP_O

In [151]:
# Ordenar y seleccionar las 14 mejores características de cada lista
top_14_america = filtered_features_list_America[:25]
top_14_turquia = filtered_features_list_Turquia[:20]
top_14_america_Rel = filtered_features_list_America_Rel[:25]
top_14_turquia_Rel = filtered_features_list_Turquia_Rel[:25]
top_14_all = filtered_features_list_all[:20]
top_14_R2 = filtered_features_list_R2[:14]

# Unir las dos listas y eliminar duplicados
union = list(set( top_14_america  + top_14_america_Rel))

print(union)


['High_subj_spec_RPD_HPC_left', 'Alpha2_canon_EPP_HPC_left', 'Alpha2_canon_RPD_HPC_left', 'High_subj_spec_EPP_PARIET_right', 'High_subj_spec_EPP_HPC_right', 'Low_subj_spec_RPD_OCC_left', 'High_subj_spec_EPP_OCC_left', 'High_subj_spec_RPD_PARIET_right', 'Alpha2_canon_EPP_PARIET_right', 'High_subj_spec_RPD_CING_left', 'High_subj_spec_RPD_HPC_right', 'Alpha2_canon_RPD_PARIET_right', 'High_subj_spec_RPD_PARIET_left', 'Low_subj_spec_RPD_HPC_right', 'Low_subj_spec_RPD_PARIET_left', 'High_subj_spec_RPD_OCC_left', 'Alpha2_canon_EPP_HPC_right', 'Alpha2_canon_RPD_CING_left', 'High_subj_spec_EPP_HPC_left', 'Low_subj_spec_RPD_PARIET_right', 'Low_subj_spec_RPD_HPC_left', 'Alpha2_canon_RPD_PARIET_left', 'High_subj_spec_EPP_CING_left', 'Alpha2_canon_EPP_PARIET_left', 'High_subj_spec_EPP_PARIET_left']
